# osmnx

In [ ]:
import osmnx as ox
import folium
import geopandas as gpd
import sys


def get_region(query: str, save_path: str | None = None) -> gpd.GeoDataFrame:
    """
    Получает границы объекта из OSM по текстовому запросу.

    :param query: текстовый запрос, например "Белгородская область, Россия"
    :param save_path: путь для сохранения GeoJSON (опционально)
    :return: GeoDataFrame с геометрией
    """
    gdf = ox.geocode_to_gdf(query)
    if gdf.empty:
        raise ValueError(f"Ничего не найдено по запросу: {query}")

    if save_path:
        gdf.to_file(save_path, driver="GeoJSON")

    return gdf


def plot_region(query: str, save_path: str | None = None, show: bool = True) -> folium.Map:
    """
    Строит интерактивную карту с folium по объекту из OSM.

    :param query: текстовый запрос, например "Белгородская область, Россия"
    :param save_path: путь для сохранения GeoJSON (опционально)
    :param show: если True — в Jupyter автоматически отобразится карта
    :return: folium.Map
    """
    gdf = get_region(query, save_path=save_path)

    centroid = gdf.geometry.iloc[0].centroid
    lat, lon = centroid.y, centroid.x

    fmap = folium.Map(location=[lat, lon], zoom_start=7, tiles="cartodbpositron")

    folium.GeoJson(
        gdf.geometry.iloc[0],
        name=query,
        tooltip=query,
        style_function=lambda x: {
            "fillColor": "blue",
            "color": "red",
            "weight": 2,
            "fillOpacity": 0.1,
        },
    ).add_to(fmap)

    # Если мы в Jupyter — показываем карту
    if show and "ipykernel" in sys.modules:
        from IPython.display import display
        display(fmap)

    return fmap


In [ ]:
query = "город Белгород"

gdf = get_region(query)
print("Границы сохранены в belgorod.geojson")

m = plot_region(query)
# m.save("belgorod.html")
print("Карта сохранена в belgorod.html")


# OSMPythonTools

In [ ]:
from OSMPythonTools.overpass import Overpass
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
import folium
import sys


def _geometry_to_polygon(geo):
    """
    Преобразует структуру geometry из Overpass в Polygon или MultiPolygon.
    """
    if not geo:
        return None

    # случай 1: список точек с lon/lat
    if isinstance(geo[0], dict) and "lon" in geo[0]:
        coords = [(p["lon"], p["lat"]) for p in geo]
        return Polygon(coords)

    # случай 2: вложенные списки (MultiPolygon)
    elif isinstance(geo[0], list):
        polygons = []
        for part in geo:
            if part and isinstance(part[0], dict):
                coords = [(p["lon"], p["lat"]) for p in part]
                polygons.append(Polygon(coords))
        return MultiPolygon(polygons)

    else:
        raise ValueError("Неизвестный формат geometry:", geo)


def get_region_overpass(name: str | int, save_path: str | None = None) -> gpd.GeoDataFrame:
    """
    Получает границы региона из OSM через Overpass API.

    :param name: название региона ("Белгородская область") или relation id (int)
    :param save_path: путь для сохранения GeoJSON (опционально)
    :return: GeoDataFrame
    """
    overpass = Overpass()

    if isinstance(name, int) or str(name).isdigit():
        query = f"relation({name});out geom;"
    else:
        query = f'relation["boundary"="administrative"]["name"="{name}"];out geom;'

    result = overpass.query(query)

    if not result.relations():
        raise ValueError(f"Регион '{name}' не найден")

    geometries = []
    for rel in result.relations():
        poly = _geometry_to_polygon(rel.geometry())
        if poly:
            geometries.append(poly)

    if not geometries:
        raise ValueError(f"Не удалось построить геометрию для '{name}'")

    geom = geometries[0] if len(geometries) == 1 else MultiPolygon(geometries)
    gdf = gpd.GeoDataFrame([{"name": str(name), "geometry": geom}], crs="EPSG:4326")

    if save_path:
        gdf.to_file(save_path, driver="GeoJSON")

    return gdf


def plot_region_overpass(name: str | int, save_path: str | None = None, show: bool = True) -> folium.Map:
    """
    Визуализирует регион через folium.
    """
    gdf = get_region_overpass(name, save_path=save_path)

    centroid = gdf.geometry.iloc[0].centroid
    lat, lon = centroid.y, centroid.x

    fmap = folium.Map(location=[lat, lon], zoom_start=7, tiles="cartodbpositron")
    folium.GeoJson(gdf.geometry.iloc[0], name=str(name), tooltip=str(name)).add_to(fmap)

    if show and "ipykernel" in sys.modules:
        from IPython.display import display
        display(fmap)

    return fmap


In [ ]:
query = "Белгородская область"

m = plot_region_overpass(query)
m
